In [0]:
from pyspark.sql.functions import col, regexp_replace, length, hour, date_format, when

df_bronze = spark.table("telecom_ete.bronze.cdr_raw")

df_silver = (
    df_bronze
    .filter(col("caller_number").isNotNull() & col("receiver_number").isNotNull())
    #.filter(length(col("caller_number")) >= 8)
    #.filter(length(col("receiver_number")) >= 8)
    .filter(col("call_duration_sec")>0)
    .withColumn("caller_number", regexp_replace(col("caller_number"), "[^0-9]",""))
    .withColumn("receiver_number", regexp_replace(col("receiver_number"), "[^0-9]",""))
    # Add "time_of_day" based on call_start_time hour
    .withColumn("hour",hour("call_start_time"))
    .withColumn("time_of_day",when (col("hour").between(5,11),"Morning")
                .when (col("hour").between(12,17),"Afternoon")
                .when (col("hour").between(18,23),"Evening")
                .otherwise("Night"))
    # Add weekend flag
    .withColumn("is_weekend",when(date_format(col("call_start_time"),"E").isin("Sat","Sun"),"Y").otherwise("N"))
)

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("call_date") \
    .saveAsTable("telecom_ete.silver.cdr_cleaned_enriched")


In [0]:
#spark.table("DSECCRIBE telecom_ete.silver.cdr_cleaned_enriched").display()

spark.sql("SHOW PARTITIONS telecom_ete.silver.cdr_cleaned_enriched").show()

+----------+
| call_date|
+----------+
|2025-05-03|
|2025-05-10|
|2025-03-09|
|2025-02-22|
|2025-03-11|
|2025-02-28|
|2025-05-19|
|2025-04-17|
|2025-02-19|
|2025-05-17|
|2025-03-28|
|2025-04-08|
|2025-02-25|
|2025-04-09|
|2025-03-25|
|2025-04-04|
|2025-05-08|
|2025-04-03|
|2025-03-27|
|2025-02-21|
+----------+
only showing top 20 rows


In [0]:
spark.sql("DESCRIBE DETAIL telecom_ete.silver.cdr_cleaned_enriched").show(truncate=False)


+------+------------------------------------+---------------------------------------+-----------+----------------------------------------------------------------------------------------------------------------------------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+-------------------------------------+----------------+----------------+-----------------------------------------+---------------------------------------------------------------+
|format|id                                  |name                                   |description|location                                                                                                                                |createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties                           |minReaderVersion|minWriterVersion|tableFeatures                            |statistics                             